In [ ]:
import os
root = './data/dermamnist'
os.makedirs(root, exist_ok=True)

# 1. 라이브러리 설치
!pip install medmnist torch torchvision --quiet

# 2. 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision import models
from medmnist import DermaMNIST

# 3. GPU 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

# 4. 데이터 전처리(transform)
transform = T.Compose([
    T.ToTensor(),
    T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

# train, validation, test 데이터셋 (224x224)
train_set = DermaMNIST(split='train', transform=transform, download=True, size=224)
val_set = DermaMNIST(split='val', transform=transform, download=True, size=224)
test_set = DermaMNIST(split='test', transform=transform, download=True, size=224)

# 5. DataLoader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)


# 6. MobileNetV2 모델 정의 및 classifier 출력 수정
model = models.mobilenet_v2(pretrained=True)
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, 7)  # DermaMNIST는 7개 클래스
model = model.to(device)

# 7. 손실함수 / optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)

# 8. 학습
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0
    running_correct = 0
    total = 0

    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).view(-1)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        running_correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = running_correct / total

    # 검증
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch {epoch:2d} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

# 9. 평가
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).view(-1)
        outputs = model(imgs)
        preds = outputs.argmax(1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Accuracy: {test_correct / test_total:.4f}")


In [ ]:
import os
root = './data/dermamnist'
os.makedirs(root, exist_ok=True)

# 1. 라이브러리 설치 (이미 설치했다면 건너뛰세요)
!pip install medmnist torch torchvision --quiet

# 2. 라이브러리 import
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.transforms as T
from torchvision import models
from medmnist import DermaMNIST
from torch.optim.lr_scheduler import ReduceLROnPlateau # Early Stopping을 위해 추가

# 3. GPU 디바이스 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device} ({torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'})")

# 4. 데이터 전처리(transform) (Data Augmentation 적용)
train_transform = T.Compose([
    T.RandomResizedCrop(224), # 이미지를 무작위로 자르고 크기 조절
    T.RandomHorizontalFlip(), # 이미지를 무작위로 좌우 반전
    T.RandomRotation(10),     # 이미지를 무작위로 회전
    T.ColorJitter(brightness=0.1, contrast=0.1, saturation=0.1, hue=0.1), # 색상 조절
    T.ToTensor(),
    T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])

# 검증 및 테스트 데이터셋에는 Data Augmentation을 적용하지 않음
val_test_transform = T.Compose([
    T.Resize(256), # 크기 조절
    T.CenterCrop(224), # 중앙 자르기
    T.ToTensor(),
    T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
])


# train, validation, test 데이터셋 (224x224) (transform 변경)
train_set = DermaMNIST(split='train', transform=train_transform, download=True, size=224)
val_set = DermaMNIST(split='val', transform=val_test_transform, download=True, size=224)
test_set = DermaMNIST(split='test', transform=val_test_transform, download=True, size=224)

# 5. DataLoader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32)
test_loader = DataLoader(test_set, batch_size=32)

# 6. MobileNetV2 모델 정의 및 classifier 출력 수정 (Dropout 추가)
model = models.mobilenet_v2(pretrained=True)
in_features = model.classifier[1].in_features

# 기존 classifier를 새로운 Sequential 레이어로 대체
model.classifier = nn.Sequential(
    nn.Dropout(p=0.2), # Dropout 레이어 추가 (p는 드롭아웃 비율, 조절 가능)
    nn.Linear(in_features, 7)  # DermaMNIST는 7개 클래스
)
model = model.to(device)

# 7. 손실함수 / optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4) # Weight Decay는 기존 코드 유지

# Early Stopping 및 Learning Rate Scheduler 설정
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
best_val_loss = float('inf')
patience_counter = 0
early_stopping_patience = 10 # 검증 손실 개선이 10번의 epoch 동안 없을 경우 중단


# 8. 학습 (Early Stopping 로직 추가)
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    model.train()
    running_loss = 0
    running_correct = 0
    total = 0

    for imgs, labels in train_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).view(-1)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        running_correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / total
    train_acc = running_correct / total

    # 검증
    model.eval()
    val_loss = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * imgs.size(0)
            val_correct += (outputs.argmax(1) == labels).sum().item()
            val_total += labels.size(0)

    val_loss /= val_total
    val_acc = val_correct / val_total

    print(f"Epoch {epoch:2d} | "
          f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

    # Early Stopping 체크
    scheduler.step(val_loss) # Learning Rate Scheduler 업데이트
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        patience_counter = 0
        # Optional: Save the best model state
        # torch.save(model.state_dict(), 'best_model.pth')
    else:
        patience_counter += 1
        if patience_counter >= early_stopping_patience:
            print(f"Early stopping at epoch {epoch}")
            break


# 9. 평가
model.eval()
test_correct = 0
test_total = 0
with torch.no_grad():
    for imgs, labels in test_loader:
        imgs = imgs.to(device)
        labels = labels.to(device).view(-1)
        outputs = model(imgs)
        preds = outputs.argmax(1)
        test_correct += (preds == labels).sum().item()
        test_total += labels.size(0)

print(f"Test Accuracy: {test_correct / test_total:.4f}")

Using device: cuda (Tesla T4)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch  1 | Train Loss: 0.9299, Train Acc: 0.6765 | Val Loss: 0.7707, Val Acc: 0.7238
Epoch  2 | Train Loss: 0.8274, Train Acc: 0.7063 | Val Loss: 0.7110, Val Acc: 0.7488
Epoch  3 | Train Loss: 0.7934, Train Acc: 0.7123 | Val Loss: 0.6823, Val Acc: 0.7537
Epoch  4 | Train Loss: 0.7719, Train Acc: 0.7207 | Val Loss: 0.7019, Val Acc: 0.7368
Epoch  5 | Train Loss: 0.7486, Train Acc: 0.7281 | Val Loss: 0.6782, Val Acc: 0.7468
Epoch  6 | Train Loss: 0.7459, Train Acc: 0.7213 | Val Loss: 0.6734, Val Acc: 0.7637
Epoch  7 | Train Loss: 0.7251, Train Acc: 0.7377 | Val Loss: 0.6535, Val Acc: 0.7707
Epoch  8 | Train Loss: 0.7187, Train Acc: 0.7383 | Val Loss: 0.6275, Val Acc: 0.7846
Epoch  9 | Train Loss: 0.6994, Train Acc: 0.7498 | Val Loss: 0.6181, Val Acc: 0.7707
Epoch 10 | Train Loss: 0.6966, Train Acc: 0.7460 | Val Loss: 0.5842, Val Acc: 0.7767
Epoch 11 | Train Loss: 0.6779, Train Acc: 0.7537 | Val Loss: 0.6072, Val Acc: 0.7727
Epoch 12 | Train Loss: 0.6707, Train Acc: 0.7548 | Val Loss: 0.60

# Task
Generate Python code for a machine learning task that includes training multiple models and combining their predictions using an ensemble technique.

## Model training

### Subtask:
Train multiple independent models (e.g., different architectures, or the same architecture with different initializations/data splits).


**Reasoning**:
Define a function to train a single model with given configurations, including data augmentation, optimizer, and scheduling, and then call this function multiple times to train different models.



In [ ]:
def train_model(config):
    """Trains a single model with the given configuration."""

    # Data Transforms (with augmentation for training)
    train_transform = T.Compose([
        T.RandomResizedCrop(224),
        T.RandomHorizontalFlip(),
        T.RandomRotation(config['augmentation_params']['rotation']),
        T.ColorJitter(
            brightness=config['augmentation_params']['brightness'],
            contrast=config['augmentation_params']['contrast'],
            saturation=config['augmentation_params']['saturation'],
            hue=config['augmentation_params']['hue']
        ),
        T.ToTensor(),
        T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
    ])

    val_test_transform = T.Compose([
        T.Resize(256),
        T.CenterCrop(224),
        T.ToTensor(),
        T.Normalize(mean=[.5, .5, .5], std=[.5, .5, .5])
    ])

    # Datasets and DataLoaders
    train_set = DermaMNIST(split='train', transform=train_transform, download=True, size=224)
    val_set = DermaMNIST(split='val', transform=val_test_transform, download=True, size=224)
    train_loader = DataLoader(train_set, batch_size=config['batch_size'], shuffle=True)
    val_loader = DataLoader(val_set, batch_size=config['batch_size'])

    # Model Initialization
    model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.IMAGENET1K_V1) # Use weights instead of pretrained
    in_features = model.classifier[1].in_features
    model.classifier = nn.Sequential(
        nn.Dropout(p=config['dropout_rate']),
        nn.Linear(in_features, 7)
    )
    model = model.to(device)

    # Loss Function and Optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=config['learning_rate'], weight_decay=config['weight_decay'])

    # Learning Rate Scheduler and Early Stopping
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=config['scheduler_factor'], patience=config['scheduler_patience'], verbose=True)
    best_val_loss = float('inf')
    patience_counter = 0

    # Training Loop
    for epoch in range(1, config['num_epochs'] + 1):
        model.train()
        running_loss = 0
        running_correct = 0 # Initialize running_correct for each epoch
        total = 0

        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device).view(-1)

            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * imgs.size(0)
            preds = outputs.argmax(1) # Get predictions
            running_correct += (preds == labels).sum().item() # Update running_correct
            total += labels.size(0)

        train_loss = running_loss / total
        train_acc = running_correct / total

        # Validation
        model.eval()
        val_loss = 0
        val_correct = 0 # Initialize val_correct for each validation
        val_total = 0
        with torch.no_grad():
            for imgs, labels in val_loader:
                imgs = imgs.to(device)
                labels = labels.to(device).view(-1)
                outputs = model(imgs)
                loss = criterion(outputs, labels)
                val_loss += loss.item() * imgs.size(0)
                preds = outputs.argmax(1) # Get predictions
                val_correct += (preds == labels).sum().item() # Update val_correct
                val_total += labels.size(0)

        val_loss /= val_total
        val_acc = val_correct / val_total

        print(f"Epoch {epoch:2d} | "
              f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

        # Early Stopping and Scheduler Step
        scheduler.step(val_loss)
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= config['early_stopping_patience']:
                print(f"Early stopping at epoch {epoch}")
                break

    return model

# Define configurations for multiple models
configs = [
    {
        'architecture': 'mobilenet_v2',
        'batch_size': 32,
        'learning_rate': 1e-3,
        'weight_decay': 1e-4,
        'num_epochs': 30, # Reduced epochs for faster execution
        'dropout_rate': 0.2,
        'augmentation_params': {'rotation': 10, 'brightness': 0.1, 'contrast': 0.1, 'saturation': 0.1, 'hue': 0.1},
        'scheduler_factor': 0.1,
        'scheduler_patience': 5,
        'early_stopping_patience': 10
    },
    {
        'architecture': 'mobilenet_v2', # Same architecture, different initialization/hyperparameters
        'batch_size': 64,
        'learning_rate': 5e-4,
        'weight_decay': 1e-3,
        'num_epochs': 30,
        'dropout_rate': 0.3,
        'augmentation_params': {'rotation': 15, 'brightness': 0.2, 'contrast': 0.2, 'saturation': 0.2, 'hue': 0.2},
        'scheduler_factor': 0.2,
        'scheduler_patience': 7,
        'early_stopping_patience': 15
    }
]

# Train multiple models
trained_models = []
for i, config in enumerate(configs):
    print(f"Training Model {i+1}...")
    model = train_model(config)
    trained_models.append(model)
    print(f"Model {i+1} Training Finished.")

Training Model 1...


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Epoch  1 | Train Loss: 0.9072, Train Acc: 0.6859 | Val Loss: 0.8682, Val Acc: 0.6600
Epoch  2 | Train Loss: 0.8182, Train Acc: 0.7003 | Val Loss: 0.7112, Val Acc: 0.7408
Epoch  3 | Train Loss: 0.7950, Train Acc: 0.7126 | Val Loss: 0.6913, Val Acc: 0.7428
Epoch  4 | Train Loss: 0.7640, Train Acc: 0.7150 | Val Loss: 0.7463, Val Acc: 0.7408
Epoch  5 | Train Loss: 0.7482, Train Acc: 0.7271 | Val Loss: 0.6701, Val Acc: 0.7498
Epoch  6 | Train Loss: 0.7233, Train Acc: 0.7337 | Val Loss: 0.6440, Val Acc: 0.7617
Epoch  7 | Train Loss: 0.7241, Train Acc: 0.7433 | Val Loss: 0.6024, Val Acc: 0.7757
Epoch  8 | Train Loss: 0.7035, Train Acc: 0.7427 | Val Loss: 0.6284, Val Acc: 0.7797
Epoch  9 | Train Loss: 0.6898, Train Acc: 0.7464 | Val Loss: 0.5813, Val Acc: 0.7916
Epoch 10 | Train Loss: 0.6699, Train Acc: 0.7624 | Val Loss: 0.5522, Val Acc: 0.8086
Epoch 11 | Train Loss: 0.6671, Train Acc: 0.7570 | Val Loss: 0.6141, Val Acc: 0.7707
Epoch 12 | Train Loss: 0.6626, Train Acc: 0.7602 | Val Loss: 0.58

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-4-1626107124.py", line 143, in <cell line: 0>
    model = train_model(config)
            ^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipython-input-4-1626107124.py", line 57, in train_model
    for imgs, labels in train_loader:
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 708, in __next__
    data = self._next_data()
           ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py", line 764, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
          

TypeError: object of type 'NoneType' has no len()